<a href="https://colab.research.google.com/github/Andhi1710/python/blob/main/(Andhi_Hamzah)_BikeShare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 Bikeshares

a) Cari tahu berapa persentase sepeda
kembali ke lokasi awalnya?

b) Biasanya berapa banyak perjalanan bikeshare
mengunjungi lebih dari sekadar titik awal / akhir,
tetapi mengunjungi beberapa poin lain sebelumnya?

# Data Import

In [1]:
!pip install kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"andhihamzah","key":"b3afadbe6663a10cdcd7032c54f70376"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d jackywang529/toronto-bikeshare-data

toronto-bikeshare-data.zip: Skipping, found more recently modified local copy (use --force to force download)


In [5]:
!ls

 drive	'kaggle (1).json'   sample_data
 files	 kaggle.json	    toronto-bikeshare-data.zip


In [6]:
import zipfile
zip_ref = zipfile.ZipFile('toronto-bikeshare-data.zip', 'r')
zip_ref.extractall('files')
zip_ref.close()

In [1]:
#memasukkan data 
import pandas as pd
import numpy as np
df1 = pd.read_csv("/content/files/bikeshare2018/bikeshare2018/Bike Share Toronto Ridership_Q1 2018.csv")
df2 = pd.read_csv("/content/files/bikeshare2018/bikeshare2018/Bike Share Toronto Ridership_Q2 2018.csv")
df3 = pd.read_csv("/content/files/bikeshare2018/bikeshare2018/Bike Share Toronto Ridership_Q3 2018.csv")
df4 = pd.read_csv("/content/files/bikeshare2018/bikeshare2018/Bike Share Toronto Ridership_Q4 2018.csv")
df2018 = df1.copy()
df2018 = df2018.append([df2, df3, df4], sort=False)
del df1
del df2
del df3
del df4


In [3]:
df2018.head()

,trip_id,trip_duration_seconds,from_station_id,trip_start_time,from_station_name,trip_stop_time,to_station_id,to_station_name,user_type
0,2383648,393,7018,1/1/2018 0:47,Bremner Blvd / Rees St,1/1/2018 0:54,7176,Bathurst St / Fort York Blvd,Annual Member
1,2383649,625,7184,1/1/2018 0:52,Ossington Ave / College St,1/1/2018 1:03,7191,Central Tech (Harbord St),Annual Member
2,2383650,233,7235,1/1/2018 0:55,Bay St / College St (West Side) - SMART,1/1/2018 0:59,7021,Bay St / Albert St,Annual Member
3,2383651,1138,7202,1/1/2018 0:57,Queen St W / York St (City Hall),1/1/2018 1:16,7020,Phoebe St / Spadina Ave,Annual Member
4,2383652,703,7004,1/1/2018 1:00,University Ave / Elm St,1/1/2018 1:12,7060,Princess St / Adelaide St E,Annual Member


In [4]:
df2018.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1922955 entries, 0 to 363489
Data columns (total 9 columns):
 #   Column                 Dtype 
---  ------                 ----- 
 0   trip_id                int64 
 1   trip_duration_seconds  int64 
 2   from_station_id        int64 
 3   trip_start_time        object
 4   from_station_name      object
 5   trip_stop_time         object
 6   to_station_id          int64 
 7   to_station_name        object
 8   user_type              object
dtypes: int64(4), object(5)
memory usage: 146.7+ MB


# BikeShare Data Toronto 2018

In [6]:
df2018.isnull().values.any() # Check if there is any missing value

False

In [7]:
#@title 1.Find out what percentage of bikes are returned to its initial location?
# prepare the dataset
df2018['time'] = pd.to_datetime(df2018.trip_start_time)
df2018.index = df2018.time.dt.date
df2018.index.name = 'index'
# cleaning the dataset
df2018 = df2018.dropna(subset=['from_station_name','to_station_name'])
df = df2018.copy()

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1922955 entries, 2018-01-01 to 2018-12-31
Data columns (total 10 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   trip_id                int64         
 1   trip_duration_seconds  int64         
 2   from_station_id        int64         
 3   trip_start_time        object        
 4   from_station_name      object        
 5   trip_stop_time         object        
 6   to_station_id          int64         
 7   to_station_name        object        
 8   user_type              object        
 9   time                   datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(5)
memory usage: 201.4+ MB


In [9]:
df

,trip_id,trip_duration_seconds,from_station_id,trip_start_time,from_station_name,trip_stop_time,to_station_id,to_station_name,user_type,time
index,,,,,,,,,,
2018-01-01,2383648,393,7018,1/1/2018 0:47,Bremner Blvd / Rees St,1/1/2018 0:54,7176,Bathurst St / Fort York Blvd,Annual Member,2018-01-01 00:47:00
2018-01-01,2383649,625,7184,1/1/2018 0:52,Ossington Ave / College St,1/1/2018 1:03,7191,Central Tech (Harbord St),Annual Member,2018-01-01 00:52:00
2018-01-01,2383650,233,7235,1/1/2018 0:55,Bay St / College St (West Side) - SMART,1/1/2018 0:59,7021,Bay St / Albert St,Annual Member,2018-01-01 00:55:00
2018-01-01,2383651,1138,7202,1/1/2018 0:57,Queen St W / York St (City Hall),1/1/2018 1:16,7020,Phoebe St / Spadina Ave,Annual Member,2018-01-01 00:57:00
2018-01-01,2383652,703,7004,1/1/2018 1:00,University Ave / Elm St,1/1/2018 1:12,7060,Princess St / Adelaide St E,Annual Member,2018-01-01 01:00:00
...,...,...,...,...,...,...,...,...,...,...
2018-12-31,4581273,379,7088,12/31/2018 23:43,Danforth Ave / Coxwell Ave,12/31/2018 23:49,7091,Donlands Station,Annual Member,2018-12-31 23:43:00
2018-12-31,4581274,306,7030,12/31/2018 23:45,Bay St / Wellesley St W,12/31/2018 23:50,7031,Jarvis St / Isabella St,Annual Member,2018-12-31 23:45:00
2018-12-31,4581275,340,7020,12/31/2018 23:49,Phoebe St / Spadina Ave,12/31/2018 23:55,7000,Fort York Blvd / Capreol Ct,Annual Member,2018-12-31 23:49:00


In [11]:
station = list(set(list(df.from_station_name.values) + list(df.to_station_name.values)))
date = df.index.unique().values

In [12]:
route = df.copy()
route = route[['from_station_name', 'to_station_name']]

In [13]:
twoway = route.copy()
twoway = twoway[twoway.from_station_name == twoway.to_station_name]
oneway = route.copy()
oneway = oneway[oneway.from_station_name != oneway.to_station_name]

In [14]:
twoway_map = twoway.groupby('from_station_name').count().sort_values(by='to_station_name', ascending=False)
print('10 Stations with the highest number of two-way traveller')
twoway_map[:100]

10 Stations with the highest number of two-way traveller


,to_station_name
from_station_name,
Bay St / Queens Quay W (Ferry Terminal),3515
Humber Bay Shores Park West,2162
York St / Queens Quay W,1969
Ontario Place Blvd / Lakeshore Blvd W,1746
Tommy Thompson Park (Leslie Street Spit),1742
...,...
Charles St W / Balmuto St - SMART,135
Elizabeth St / Edward St (Bus Terminal),132
The Royal Ontario Museum (Bloor St Entrance),130


In [17]:
#@title 2.How many bikeshare trips usually visit more than just start/end points, but visited some other points before the end of the day?


In [15]:
# mapping the number of outgoing bike from each station each day in 2017
outmap = pd.get_dummies(route.from_station_name).groupby('index').sum()
# mapping the number of incoming bike to each station each day in 2017
inmap = pd.get_dummies(route.to_station_name).groupby('index').sum()

In [16]:
outmap.head(5) # number of bikes leaves the station

,111 Bond St (North of Dundas St E) - SMART,12 Harbour St,123 Queen St W,161 Bleecker St (South of Wellesley),20 Charles St E,200 Bloor St. E.,25 York St  Union Station South,424 Wellington St W,457 King St. W. at Spadina,51 Parliament St,519 Church St - SMART,80 Clinton St (North of College),Adelaide St W / Bay St - SMART,Alma Ave / Gladstone Ave SMART,Alton Ave / Dundas St E (Greenwood Park),Amelia St. / Sumach St.,Augusta Ave / Denison Sq,Augusta Ave / Dundas St W,Baldwin Ave / Spadina Ave - SMART,Balliol St / Yonge St SMART,Bathurst St / Adelaide St W,Bathurst St / Dundas St W,Bathurst St / Fort York Blvd,Bathurst St / Lennox St,Bathurst St/Queens Quay(Billy Bishop Airport),Bathurst Subway Station,Bay St / Albert St,Bay St / Charles St - SMART,Bay St / College St (East Side),Bay St / College St (West Side) - SMART,Bay St / Davenport Rd,Bay St / Queens Quay W (Ferry Terminal),Bay St / St. Joseph St,Bay St / Wellesley St W,Beltline Trail / Chaplin Cres.,Beverley St / Dundas St W,Beverley St / College St,Blake St / Boultbee Ave,Bloor GO / UP Station (West Toronto Railpath),Bloor St E / Huntley St - SMART,...,The Royal Ontario Museum (Bloor St Entrance),Thompson St / Broadview Ave - SMART,Tommy Thompson Park (Leslie Street Spit),Toronto Eaton Centre (Yonge St),Trinity St /Front St E,Ulster St / Bathurst St,Union Station,University Ave / College St (West),University Ave / Elm St,University Ave / Gerrard St W,Vanauley St / Queen St W - SMART,Victoria Park Subway Station - SMART,Wallace Ave / Symington Ave - SMART,Walton St / Elizabeth St - SMART,Ward Ave / Wallace Ave,Wellesley St / Queen's Park Cres,Wellesley St E / Yonge St (Green P),Wellington St W / Bay St,Wellington St W / Portland St,Wellington St W / Stafford St,Wells Hill Ave / St Clair Ave W,Westmoreland Ave / Fernbank Ave,Widmer St / Adelaide St W,Willcocks St / St. George St,Winona Dr / Davenport Rd,Wolfpack - 7219,Wright / Sorauren (Sorauren Park) - SMART,Wychwood Ave / Benson Ave - SMART,Yarmouth Rd / Christie St,Yonge St / Alexander St - SMART,Yonge St / Aylmer Ave,Yonge St / Bloor St,Yonge St / Davisville Ave,Yonge St / Dundas Sq,Yonge St / Dundonald St - SMART,Yonge St / Harbour St,Yonge St / Wood St,Yonge St / Yorkville Ave,York St / King St W - SMART,York St / Queens Quay W
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-01-01,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,2,3,3,1,2,0,4,2,1,2,3,3,0,2,1,0,0,0,...,3,1,0,1,0,2,5,1,2,2,0,0,0,0,1,1,1,1,0,0,0,0,1,0,0,0,0,0,3,2,0,0,0,0,0,1,0,0,0,0
2018-01-02,1,0,0,1,0,0,3,3,0,4,3,4,6,0,0,0,2,0,5,0,0,6,8,2,4,2,11,1,11,10,1,6,7,25,0,8,4,0,4,2,...,3,2,0,6,1,1,16,11,8,12,2,0,2,2,3,1,5,4,5,1,0,0,13,5,0,0,0,1,0,4,0,0,0,0,2,4,5,4,5,8
2018-01-03,1,0,0,2,0,0,4,3,0,5,8,2,4,0,0,0,4,4,13,0,0,6,9,0,5,2,12,3,22,16,1,4,15,27,0,7,8,0,5,2,...,3,0,0,4,1,1,15,10,6,16,2,0,1,8,1,5,9,6,6,0,0,0,14,7,0,0,0,0,2,6,1,1,0,0,5,3,4,7,6,7
2018-01-04,0,0,0,0,0,0,8,2,0,3,8,1,6,0,0,0,3,4,7,0,0,8,9,2,6,2,10,4,11,9,1,8,16,19,0,10,14,0,4,2,...,6,1,0,5,1,2,24,8,8,11,2,0,3,5,1,9,8,4,3,0,0,0,15,8,0,0,0,0,2,5,0,0,0,0,2,5,6,3,9,8
2018-01-05,0,0,0,0,0,0,1,2,0,2,4,2,1,0,0,0,6,2,8,0,0,8,10,2,6,3,5,3,8,8,1,1,10,18,0,10,7,0,3,0,...,5,0,0,2,1,1,14,2,5,7,1,0,2,2,0,3,4,4,2,1,0,0,6,12,0,0,0,1,0,4,1,1,0,0,2,5,4,2,5,5


In [17]:
inmap.head(5) # number of bikes entering the station

,111 Bond St (North of Dundas St E) - SMART,12 Harbour St,123 Queen St W,161 Bleecker St (South of Wellesley),20 Charles St E,200 Bloor St. E.,25 York St  Union Station South,424 Wellington St W,457 King St. W. at Spadina,51 Parliament St,519 Church St - SMART,80 Clinton St (North of College),Adelaide St W / Bay St - SMART,Alma Ave / Gladstone Ave SMART,Alton Ave / Dundas St E (Greenwood Park),Amelia St. / Sumach St.,Augusta Ave / Denison Sq,Augusta Ave / Dundas St W,Baldwin Ave / Spadina Ave - SMART,Balliol St / Yonge St SMART,Bathurst St / Adelaide St W,Bathurst St / Dundas St W,Bathurst St / Fort York Blvd,Bathurst St / Lennox St,Bathurst St/Queens Quay(Billy Bishop Airport),Bathurst Subway Station,Bay St / Albert St,Bay St / Charles St - SMART,Bay St / College St (East Side),Bay St / College St (West Side) - SMART,Bay St / Davenport Rd,Bay St / Queens Quay W (Ferry Terminal),Bay St / St. Joseph St,Bay St / Wellesley St W,Beltline Trail / Chaplin Cres.,Beverley St / Dundas St W,Beverley St / College St,Blake St / Boultbee Ave,Bloor GO / UP Station (West Toronto Railpath),Bloor St E / Huntley St - SMART,...,The Royal Ontario Museum (Bloor St Entrance),Thompson St / Broadview Ave - SMART,Tommy Thompson Park (Leslie Street Spit),Toronto Eaton Centre (Yonge St),Trinity St /Front St E,Ulster St / Bathurst St,Union Station,University Ave / College St (West),University Ave / Elm St,University Ave / Gerrard St W,Vanauley St / Queen St W - SMART,Victoria Park Subway Station - SMART,Wallace Ave / Symington Ave - SMART,Walton St / Elizabeth St - SMART,Ward Ave / Wallace Ave,Wellesley St / Queen's Park Cres,Wellesley St E / Yonge St (Green P),Wellington St W / Bay St,Wellington St W / Portland St,Wellington St W / Stafford St,Wells Hill Ave / St Clair Ave W,Westmoreland Ave / Fernbank Ave,Widmer St / Adelaide St W,Willcocks St / St. George St,Winona Dr / Davenport Rd,Wolfpack - 7219,Wright / Sorauren (Sorauren Park) - SMART,Wychwood Ave / Benson Ave - SMART,Yarmouth Rd / Christie St,Yonge St / Alexander St - SMART,Yonge St / Aylmer Ave,Yonge St / Bloor St,Yonge St / Davisville Ave,Yonge St / Dundas Sq,Yonge St / Dundonald St - SMART,Yonge St / Harbour St,Yonge St / Wood St,Yonge St / Yorkville Ave,York St / King St W - SMART,York St / Queens Quay W
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-01-01,0,0,0,0,0,0,1,0,0,0,1,1,1,0,0,0,1,0,2,0,0,1,4,3,3,0,2,0,3,4,1,5,2,3,0,2,1,0,0,0,...,1,0,0,1,0,2,6,2,1,1,1,0,0,0,0,2,1,1,1,0,0,0,3,0,0,0,0,0,3,1,0,0,0,0,0,0,2,1,0,0
2018-01-02,0,0,0,0,0,0,5,3,0,4,4,1,8,0,0,0,3,1,3,0,0,8,7,0,4,0,11,1,16,3,0,8,6,14,0,8,3,0,6,3,...,3,2,0,7,1,1,27,7,10,7,3,0,0,5,4,2,2,11,6,0,0,0,14,2,0,0,2,0,1,5,2,0,0,0,2,4,5,0,4,8
2018-01-03,0,0,0,1,0,0,8,2,0,1,8,0,9,0,0,0,5,1,9,0,0,8,6,1,5,2,11,0,24,11,1,7,15,18,0,10,8,0,3,0,...,7,2,0,1,1,0,32,7,7,11,4,0,1,8,0,5,7,16,7,0,0,0,13,5,0,0,0,0,1,3,1,2,0,0,2,4,5,5,8,11
2018-01-04,1,0,0,1,0,0,7,2,0,3,3,2,8,0,0,0,1,6,3,0,0,6,10,2,7,1,13,2,15,7,1,10,13,12,0,8,12,0,5,4,...,11,2,0,4,4,0,32,10,4,13,3,0,1,6,1,10,9,12,4,0,0,0,14,6,0,0,0,0,0,4,2,0,0,0,1,7,6,3,6,7
2018-01-05,0,0,0,0,0,0,7,5,0,2,4,0,6,0,0,0,1,2,5,0,0,7,8,2,11,1,7,1,8,5,0,3,8,11,0,8,7,0,5,2,...,3,1,0,1,1,1,23,4,9,8,1,0,2,4,0,7,3,10,4,1,0,0,5,8,0,0,0,0,1,5,0,0,0,0,3,4,2,0,1,6


In [18]:
print('number of station with enough bike to use next morning, aka number of bikes entering > number of bikes leaving the station')
((inmap - outmap)>=0).sum(axis=1)

number of station with enough bike to use next morning, aka number of bikes entering > number of bikes leaving the station


index
2018-01-01    359
2018-01-02    359
2018-01-03    359
2018-01-04    359
2018-01-05    359
             ... 
2018-12-27    359
2018-12-28    359
2018-12-29    359
2018-12-30    359
2018-12-31    359
Length: 365, dtype: int64

In [19]:
print('number of station with less bike to use next morning, or need a crew to return bikes back to station before next morning')
((inmap - outmap)<0).sum(axis=1)

number of station with less bike to use next morning, or need a crew to return bikes back to station before next morning


index
2018-01-01    0
2018-01-02    0
2018-01-03    0
2018-01-04    0
2018-01-05    0
             ..
2018-12-27    0
2018-12-28    0
2018-12-29    0
2018-12-30    0
2018-12-31    0
Length: 365, dtype: int64

In [20]:
print('Station and the total number of days in 2018 where stations need more bikes to be returned by the crew every night')
((inmap - outmap)<0).sum(axis=0).sort_values(ascending=False)[:20]

Station and the total number of days in 2018 where stations need more bikes to be returned by the crew every night


York St / Queens Quay W                       0
Front St E / Cherry St                        0
Dundas St W / Roncesvalles Green P - SMART    0
Dundas St W / St. Patrick St                  0
Dundas St W / Yonge St                        0
Dundonald St / Church St                      0
Dupont St / Emerson Ave SMART                 0
East Liberty St / Pirandello St               0
Eastern Ave / Winnifred Ave                   0
Edward St / Yonge St                          0
Eglinton Ave W / Henning Ave SMART            0
Elizabeth St / Edward St (Bus Terminal)       0
Erskine Ave / Yonge St SMART                  0
Essex St / Christie St                        0
Euclid Ave / Bloor St W                       0
Exhibition GO (Atlantic Ave)                  0
Exhibition GO Station                         0
Fort York  Blvd / Capreol Ct                  0
Fort York Blvd / Bathurst St SMART            0
Fort York Blvd / Garrison Rd                  0
dtype: int64

In [21]:
bike_minus = inmap - outmap # incoming bikes minus leaving bikes
bike_minus = np.absolute(bike_minus[bike_minus < 0]) # show only minus value

In [22]:
bike_minus.head(10) # number of bikes that required by crew to be returned to each station

,111 Bond St (North of Dundas St E) - SMART,12 Harbour St,123 Queen St W,161 Bleecker St (South of Wellesley),20 Charles St E,200 Bloor St. E.,25 York St  Union Station South,424 Wellington St W,457 King St. W. at Spadina,51 Parliament St,519 Church St - SMART,80 Clinton St (North of College),Adelaide St W / Bay St - SMART,Alma Ave / Gladstone Ave SMART,Alton Ave / Dundas St E (Greenwood Park),Amelia St. / Sumach St.,Augusta Ave / Denison Sq,Augusta Ave / Dundas St W,Baldwin Ave / Spadina Ave - SMART,Balliol St / Yonge St SMART,Bathurst St / Adelaide St W,Bathurst St / Dundas St W,Bathurst St / Fort York Blvd,Bathurst St / Lennox St,Bathurst St/Queens Quay(Billy Bishop Airport),Bathurst Subway Station,Bay St / Albert St,Bay St / Charles St - SMART,Bay St / College St (East Side),Bay St / College St (West Side) - SMART,Bay St / Davenport Rd,Bay St / Queens Quay W (Ferry Terminal),Bay St / St. Joseph St,Bay St / Wellesley St W,Beltline Trail / Chaplin Cres.,Beverley St / Dundas St W,Beverley St / College St,Blake St / Boultbee Ave,Bloor GO / UP Station (West Toronto Railpath),Bloor St E / Huntley St - SMART,...,The Royal Ontario Museum (Bloor St Entrance),Thompson St / Broadview Ave - SMART,Tommy Thompson Park (Leslie Street Spit),Toronto Eaton Centre (Yonge St),Trinity St /Front St E,Ulster St / Bathurst St,Union Station,University Ave / College St (West),University Ave / Elm St,University Ave / Gerrard St W,Vanauley St / Queen St W - SMART,Victoria Park Subway Station - SMART,Wallace Ave / Symington Ave - SMART,Walton St / Elizabeth St - SMART,Ward Ave / Wallace Ave,Wellesley St / Queen's Park Cres,Wellesley St E / Yonge St (Green P),Wellington St W / Bay St,Wellington St W / Portland St,Wellington St W / Stafford St,Wells Hill Ave / St Clair Ave W,Westmoreland Ave / Fernbank Ave,Widmer St / Adelaide St W,Willcocks St / St. George St,Winona Dr / Davenport Rd,Wolfpack - 7219,Wright / Sorauren (Sorauren Park) - SMART,Wychwood Ave / Benson Ave - SMART,Yarmouth Rd / Christie St,Yonge St / Alexander St - SMART,Yonge St / Aylmer Ave,Yonge St / Bloor St,Yonge St / Davisville Ave,Yonge St / Dundas Sq,Yonge St / Dundonald St - SMART,Yonge St / Harbour St,Yonge St / Wood St,Yonge St / Yorkville Ave,York St / King St W - SMART,York St / Queens Quay W
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [23]:
print('20 Stations with the highest number of required returned bikes in a day')
np.max(bike_minus, axis=0).sort_values(ascending=False)[:20]

20 Stations with the highest number of required returned bikes in a day


111 Bond St (North of Dundas St E)  - SMART   NaN
12 Harbour St                                 NaN
123 Queen St W                                NaN
161 Bleecker St (South of Wellesley)          NaN
20 Charles St E                               NaN
200 Bloor St. E.                              NaN
25 York St   Union Station South             NaN
424 Wellington St W                           NaN
457 King St. W. at Spadina                    NaN
51 Parliament St                              NaN
519 Church St - SMART                         NaN
80 Clinton St (North of College)              NaN
Adelaide St W / Bay St - SMART                NaN
Alma Ave / Gladstone Ave SMART                NaN
Alton Ave / Dundas St E (Greenwood Park)      NaN
Amelia St. / Sumach St.                       NaN
Augusta Ave / Denison Sq                      NaN
Augusta Ave / Dundas St W                     NaN
Baldwin Ave / Spadina Ave - SMART             NaN
Balliol St / Yonge St  SMART                  NaN


In [24]:
f1=df['trip_duration_seconds']<301

In [25]:
f1

index
2018-01-01    False
2018-01-01    False
2018-01-01     True
2018-01-01    False
2018-01-01    False
              ...  
2018-12-31    False
2018-12-31    False
2018-12-31    False
2018-12-31    False
2018-12-31    False
Name: trip_duration_seconds, Length: 1922955, dtype: bool